In [91]:
import h5py
import torch
import ifcb
from ifcb.data.adc import SCHEMA_VERSION_1
from ifcb.data.stitching import InfilledImages
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets
import numpy as np
import imageio

# Getting classes names

In [99]:
model_path = 'training-output/model.pt'
model_dict = torch.load(model_path)
classes = model_dict['classes']

# Getting roi_numbers and scores

In [132]:
filename = 'run-output/ExampleRunResults/D20200605T214358_IFCB010_class_v2.h5'
with h5py.File(filename, "r") as f:
    # List all groups
    keys = list(f.keys())
    print("Keys: %s" % keys)
    # Get the data
    roi_numbers = list(f[keys[0]])
    scores = list(f[keys[1]])

Keys: ['roi_numbers', 'scores']


# Converting raw data to images

In [134]:
pid = 'D20200605T214358_IFCB010'
data_path = 'run-data'
output_path = 'run-output/ExampleRunResults'
data_dir = ifcb.DataDirectory(data_path)

In [ ]:
for folder_name in classes:
    path = output_path + '/' + folder_name
    try:
        os.mkdir(path)
    except OSError:
        print ("Creation of the directory %s failed" % path)
    else:
        print ("Successfully created the directory %s " % path)

In [137]:
for sample_bin in data_dir:
    n_images = len(sample_bin.images)
    lid = sample_bin.lid
    print('{} has {} image(s)'.format(lid, n_images))
    with sample_bin:
        for roi_number in sample_bin.images:
            pos = roi_numbers.index(roi_number)
            best_class = np.where(scores[pos] == np.amax(scores[pos]))[0][0]
            folder_name = classes[best_class]
            img_path = output_path + '/' + folder_name + '/' +  pid + '_' + str(roi_number) + '.png'
            imageio.imwrite(img_path, sample_bin.images[roi_number]) 

D20200605T214358_IFCB010 has 6723 image(s)
